# How to: Cold load sweep analysis 
How to analyze a cold load sweep data set using the velma branch of nistqsptdm repository.  Data stored in a .json file in custom data format IVColdloadSweepData within iv_data.py.  A data map csv file is also required to map readout channels to detector names.

Uses file example_data/example_IVColdloadSweepData.json.
This file is the same as /data/uber_omt/20231013/taurus_ColumnA_ivs_20231114_1699985874.json



In [1]:
# the python modules
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget 

import detchar
import detchar.analysis.ivAnalysis_utils as ivut
from detchar.analysis.detector_map import DetectorMap

In [2]:
file = 'example_data/example_IVColdloadSweepData.json' 
detector_map_file = 'example_data/dmap_IVColdloadSweepData.csv'


In [3]:
dm = DetectorMap(detector_map_file) # instance of detector map file
ivcl = ivut.IVColdloadSweepAnalyzer(file,dm) # main class for analysis of an IV cold load sweep

In [4]:
# helper functions
def get_predicted_power_for_row(row_index,cl_indices):
    ''' return top-hat-band predicted power to be used to determine optical efficiency '''
    [fstart,fend] = dm.map_dict['Row%02d'%row_index]['freq_edges_ghz']
    predicted_power_w = []
    for t in np.array(ivcl.post_cl_temps_k)[cl_indices,0]:
        predicted_power_w.append(ivut.thermalPower(fstart*1e9,fend*1e9,t))
    return predicted_power_w

## Check environmental conditions
What coldload and bath temperatures does this data file have?  Where these commanded values acheived?

In [5]:
ivcl.print_info()

In [6]:
plt.figure()
ivcl.plot_measured_bath_temps()

In [7]:
#plt.figure()
ivcl.plot_measured_cl_temps() # there are two thermometers on the load "ChA" and "ChB"

In [8]:
ivcl.pre_cl_temps_k # explicitly list the measured coldload temperatures

## check detector data quality

In [9]:
# change index to see data quality for a particular detector
ivcl.plot_cl_temp_sweep_for_row(row_index=1,bath_temp_index=0) 

# Do full data analysis on a single channel to extract the optical efficiency

In [10]:
row_index = 4
cl_indices = list(range(1,8))
predicted_power_w = get_predicted_power_for_row(row_index,cl_indices)

In [11]:
eta_single_row = ivcl.plot_sweep_analysis_for_row(row_index,0,cl_indices,predicted_power_w=np.array(predicted_power_w))

In [12]:
eta_single_row.eta_Dp_arr # fixed temperature reference.  

In [13]:
np.shape(eta_single_row.eta_Dp_arr) # N_resistance_fractions x M-1 coldload temperatures

In [14]:
print('Optical efficiency (fixed ref method) of Row%02d is %.3f +/- %0.3f'%(row_index,np.mean(eta_single_row.eta_Dp_arr),np.std(eta_single_row.eta_Dp_arr)))

In [15]:
eta_single_row.eta_dp_arr # eta_dp_arr (not little d) is the differential method

In [16]:
print('Optical efficiency (diff method) of Row%02d is %.3f +/- %0.3f'%(row_index,np.mean(eta_single_row.eta_dp_arr),np.std(eta_single_row.eta_Dp_arr)))